In [1]:
#importing all the libraries:
import pandas as pd
import numpy as np
import time
import pandas_gbq
from google.cloud import bigquery
import os,os.path
from xmlrpc import client
import db_dtypes
import re

In [2]:
class duplicate_v1:
    def __init__(self):
        self.j_file_path = "examly-events-26d60ca289a0.json"
        self.cleaner = re.compile('<.*?>|&([a-z0-9]+|#[0-9]{1,6}|#x[0-9a-f]{1,6});')
        
    def connect_bigquerry(self):
        start_time = time.time()
        print(f"Connecting the bigquerry...")
        os.environ['GOOGLE_APPLICATION_CREDENTIALS']=os.path.expanduser(self.j_file_path)
        print(f"Connected to the bigquerry...\nTime taken :: {time.time() - start_time}")
        
        
    def fetch_data(self,query:str):
        start_time = time.time()
        print(f"Fetching data from the bigquerry...")
        # Fetching the whole data
        client = bigquery.Client()
        whole_data = pandas_gbq.read_gbq(query, use_bqstorage_api=True,dialect="standard")
        print(f"Data from bigquery downloaded...\nTime taken :: {time.time() - start_time}")
        return whole_data
    
    
    def col_to_drop(self,columns:list):
        return self.data.drop(columns,axis=1)
    
    def select_data(self,whole_data,q_type:str):
        return whole_data.iloc[np.where(whole_data["question_type"] == q_type)]
    
    def insert_col(self,data,pos,col_name,value):
        return data.insert(pos,col_name,value)
    
    
    def clean_data(self,data):
        start_time = time.time()
        data["question_data"] = data["question_data"].replace(r"<p></p>",np.NaN,regex=True)
        if not "clean_question_data" in data.columns:
            data.insert(loc=3,column="clean_question_data",value=data["question_data"].str.replace(r'<[^<>]*>','',regex=True).replace('&amp','').replace('&nbsp','').replace('}',''))
        #data["clean_question_data"] = data["question_data"].str.replace(r'<[^<>]*>','',regex=True)
        print(f"Data cleaned...\nTime taken :: {time.time() - start_time}")
        return data
    
     

    def cleanhtml(self,raw_html):
        cleantext = re.sub(self.cleaner, '', raw_html)
        return cleantext
    
    def clean_answer_data(self,data):
        start_time = time.time()
        if "cleaned_mcq_questions_options" in list(data.columns):
            none_idx = []
            print(f"Column present....")
            for i in range(len(data["mcq_questions_options"])):
                sample = data["mcq_questions_options"].iloc[i]
                if not sample == None:
                    size = len(eval(list([sample])[0]))
                    data_ans = []
                    for n in range(size):
                        data_ans.append(self.cleanhtml(list(eval(list([sample])[0])[n].values())[0]))
                    data["cleaned_mcq_questions_options"].iloc[i] = data_ans
                else:
                    none_idx.append(i)
            print(len(none_idx))
            print(f"Answer Data cleaned...\nTime taken :: {time.time() - start_time}")
            return data
        else:
            print("Insert cleaned_mcq_questions_options into your data...")
    
    
    def clean_nan(self,data):
        data = data.drop(index=list(np.where(data["clean_question_data"] == "nan")[0]))
        return data
    
    
    def transform_data(self,cleaned_data):
        start_time = time.time()
        data_new = cleaned_data
        data_new["clean_question_data"] = [str(data_new["clean_question_data"].iloc[i]).strip().lower() for i in range(len(data_new["clean_question_data"]))]
        return data_new
    
    
    def filter_duplicate(self,transformed_data):
        start_time = time.time()
        data = transformed_data
        data["duplicate"] = data["clean_question_data"].duplicated()
        duplicated = data.iloc[np.where(data["duplicate"] == True)]
        return duplicated.reset_index()
    
    
    def find_dup_idx(self,data,scentence:str) -> list:
        return list(np.where(scentence == data["clean_question_data"][::1])[0])
    
    
    # to show the variations:
    def variations(self,filtered_data,idx:list):
        start_time = time.time()
        if len(filtered_data.iloc[idx].index)>1:
            print(f"filtered_data :: {len(filtered_data.iloc[idx].index)}")
            q_type = int(len(filtered_data["question_type"].iloc[idx]))
            #q_type = filtered_data["question_type"].nunique()
            if q_type > 2:
                print(f"q_type :: {q_type}")
                q_sub = int(len(filtered_data["subject"].iloc[idx]))
                #q_sub = filtered_data["subject"].nunique()
                if q_sub > 2:
                    print(f"q_sub :: {q_sub}")
                    q_topic = int(len(filtered_data["topic"].iloc[idx]))
                    #q_topic = filtered_data["topic"].nunique()
                    if q_topic > 2:
                        print(f"q_topic :: {q_topic}")
                        q_sub_topic = int(len(filtered_data["sub_topic"].iloc[idx]))
                        #q_sub_topic = filtered_data["sub_topic"].nunique()
                        if q_sub_topic > 2:
                            print(f"q_sub_topic :: {q_sub_topic}")
                            print(f"Variation found...\nTime taken :: {time.time() - start_time}")
                            return filtered_data.iloc[idx]
        else:
            print(f"No variations found...\nTime taken :: {time.time() - start_time}")

In [3]:
query = '''
SELECT
*,
(select qb_name from `examly-events.examly_warehouse_mysql_replica.question_banks` where qb_id = dup.qb_id) as qb_name
FROM `examly-events.schema_testing_views.questions_dup_view` dup
'''

duplicate = duplicate_v1()

In [4]:
# connecting to the bigquerry:
duplicate.connect_bigquerry()

Connecting the bigquerry...
Connected to the bigquerry...
Time taken :: 7.200241088867188e-05


In [5]:
# fetching the data:
whole_data = duplicate.fetch_data(query)

Fetching data from the bigquerry...


Downloading: 100%|██████████| 9279104/9279104 [00:15<00:00, 598125.78rows/s]


Data from bigquery downloaded...
Time taken :: 75.81802821159363


In [38]:
q_type = "mcq_single_correct"
mcq_single_correct_data = duplicate.select_data(whole_data,q_type)

In [39]:
mcq_single_correct_data.head()

,qb_id,q_id,question_data,manual_difficulty,question_type,imported,subject_id,topic_id,sub_topic_id,subject,topic,sub_topic,createdAt,school_id,q_deletedAt,qb_deletedAt,school_code,mcq_questions_options,mcq_questions_answer,qb_name
0,c3e6cf7c-393d-4e51-b2da-c5fa81efaa8f,c47ef293-5193-4910-a92a-8195509ddb90,"<p><span style=""color: rgb(0, 0, 10);"">Digital...",Easy,mcq_single_correct,original_question,6ed7b6f1-be79-494b-b488-bbc4a2156f95,43de6b0e-c7fe-45ca-9e42-f5cc9120b15f,95641ed5-1986-401a-b3b9-a21ac54a3f38,COE,UNIT1,CO1,2021-10-25 20:04:59,c7bde760-cc4a-4d2d-a91d-0fe6031a8ff1,NaT,NaT,psgtech850,"[{""text"":""<p><span style=\""color: rgb(0, 0, 10...","{""args"":[""<p><span style=\""color: rgb(0, 0, 10...",BACKUP_QB_15I302/19I302 COMPUTER ARCHITECTURE_...
1,530f79d2-fa66-4ef7-9ea1-e083a6504946,a8257d80-a435-45b9-a67a-a11aace7147e,"<p><span style=""color: rgb(0, 0, 10);"">_______...",Easy,mcq_single_correct,original_question,c1ecede5-ce85-404a-b086-211fcf0de76e,2fc15da6-d65b-4055-b022-ac262bc9486a,0c911d14-73dc-4043-95c0-c82566e245cf,End Semester,Unit 5,CO5,2021-03-11 05:57:51,bc65162a-72ff-496a-bd14-7f23335ea6f0,NaT,NaT,rmkec496,"[{""text"":""<p><span style=\""color: rgb(0, 0, 10...","{""args"":[""<p><span style=\""color: rgb(0, 0, 10...",20EL102_BCVS1_CB_ES_AT_PartB
2,be4f0063-bcb0-4899-a94f-628727a77417,add83be4-c92e-40e6-a947-b7e5f5ced412,"<p><span style=""color: rgb(0, 0, 10);"">T</span...",Hard,mcq_single_correct,original_question,6ed7b6f1-be79-494b-b488-bbc4a2156f95,43de6b0e-c7fe-45ca-9e42-f5cc9120b15f,95641ed5-1986-401a-b3b9-a21ac54a3f38,COE,UNIT1,CO1,2021-10-30 06:58:01,c7bde760-cc4a-4d2d-a91d-0fe6031a8ff1,NaT,NaT,psgtech850,"[{""text"":""<p><span style=\""color: rgb(0, 0, 10...","{""args"":[""<p><span style=\""color: rgb(0, 0, 10...",QB_18FD53 APPAREL PRODUCTION PLANNING AND CONT...
3,3da85cdd-3a7d-4d71-870f-a18cf8a207e7,adb3f306-9069-488c-802d-2da195ec20c3,"<p>Assume double[][] x = new double[4][5], wha...",Medium,mcq_single_correct,unlimited_purchase,9f91343b-be0e-4336-b199-30d823a58046,bdbf4244-4656-4c89-a7e7-4ed44b06d8fb,4d31960b-1a52-41d3-83f6-13879e900e2b,MCQ,C Programming - Array,Multidimensional array,2020-07-25 09:31:38,1e219353-c3bf-48b0-9825-e17ff94ec4f2,NaT,NaT,abes819,"[{""text"":""<p>4 and 4</p>"",""media"":""""},{""text"":...","{""args"":[""<p>4 and 5</p>""],""partial"":[]}",HoP_Technical Test 1
4,e97ed478-5fa7-486d-92e0-84db45c0efb7,85581f98-d20d-4676-ab24-2a78967e57cb,"<p><span style=""color: rgb(0, 0, 10);"">Write a...",Medium,mcq_single_correct,original_question,6ed7b6f1-be79-494b-b488-bbc4a2156f95,4f7eb276-bbfa-446c-bcbc-d340a5559958,fa055fce-aca3-46a8-bfc2-2e3fe47fc228,COE,UNIT2,CO2,2021-10-26 09:48:32,c7bde760-cc4a-4d2d-a91d-0fe6031a8ff1,NaT,2021-10-26 11:19:22,psgtech850,"[{""text"":""<p><span style=\""color: rgb(0, 0, 10...","{""args"":[""<p><span style=\""color: rgb(0, 0, 10...",NOV21_19P501_COMPUTER NUMERICAL CONTROL MACHINES


In [40]:
mcq_single_correct_data.shape

(8567512, 20)

In [41]:
idx = [] # empty mcq rows whose question deletion or qb deletion data is absent.
for i in range(len(mcq_single_correct_data["question_data"])):
    if mcq_single_correct_data["q_deletedAt"].iloc[i] is pd.NaT:
        if mcq_single_correct_data["qb_deletedAt"].iloc[i] is pd.NaT:
            if mcq_single_correct_data["mcq_questions_options"].iloc[i] == '[]':
                idx.append(i)

In [42]:
len(idx)

2320

In [43]:
mcq_single_correct_data.iloc[idx]

,qb_id,q_id,question_data,manual_difficulty,question_type,imported,subject_id,topic_id,sub_topic_id,subject,topic,sub_topic,createdAt,school_id,q_deletedAt,qb_deletedAt,school_code,mcq_questions_options,mcq_questions_answer,qb_name
55231,419cd94b-ad18-4580-9dc0-b7c4bf825e11,3d7713f4-be70-43c6-9a1a-589dc991a292,<p>}</p>,Medium,mcq_single_correct,original_question,fc9d8672-4f7a-4800-8f05-294dc460feda,b7425c21-2670-4bcd-ada4-24847c8c9408,55021f2e-e729-4ade-99d1-f8d14879e041,bulk-imported,bulk-imported,bulk-imported,2018-08-30 14:22:39,6ff9556e-0409-48a2-b834-9f740f56a5e6,NaT,NaT,talentio627,[],{},Capgemini QB 4 - technical
60640,daa576d4-93dc-4dad-b6b9-80ab2f26afab,521b9f3e-5071-46cb-9ab0-c68cd45bf722,<p>} }</p>,Medium,mcq_single_correct,original_question,fc9d8672-4f7a-4800-8f05-294dc460feda,b7425c21-2670-4bcd-ada4-24847c8c9408,55021f2e-e729-4ade-99d1-f8d14879e041,bulk-imported,bulk-imported,bulk-imported,2018-09-05 09:04:05,6ff9556e-0409-48a2-b834-9f740f56a5e6,NaT,NaT,talentio627,[],{},Capgemini QB 5- Technical
157800,419cd94b-ad18-4580-9dc0-b7c4bf825e11,18befd39-f8fb-479c-8365-b98044e3588e,<p>return 0; }</p>,Medium,mcq_single_correct,original_question,fc9d8672-4f7a-4800-8f05-294dc460feda,b7425c21-2670-4bcd-ada4-24847c8c9408,55021f2e-e729-4ade-99d1-f8d14879e041,bulk-imported,bulk-imported,bulk-imported,2018-08-30 14:22:38,6ff9556e-0409-48a2-b834-9f740f56a5e6,NaT,NaT,talentio627,[],{},Capgemini QB 4 - technical
159535,8aee3d98-c276-4065-9b01-599c75cae58e,2faa8263-b96e-437e-84ef-46fa7b01cced,<p></p>,Medium,mcq_single_correct,original_question,None,None,None,None,None,None,2020-08-25 18:44:30,3e4ea61d-dca6-4ec8-94d9-f5f632878764,NaT,NaT,siet826,[],{},MPAT Hexaware 26.08.2020
162440,966b29e0-0ceb-4f1f-af59-02acf4f8a5e9,f33ccc65-f8d3-4203-aee2-f1f55c992009,<p></p>,Medium,mcq_single_correct,original_question,None,None,None,None,None,None,2020-10-16 05:18:00,3e4ea61d-dca6-4ec8-94d9-f5f632878764,NaT,NaT,siet826,[],{},CTS Topic-Wise QP 16.10.2020
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
8974511,4d5534ab-4fd5-4f85-a5f7-0ca9fe32a803,82a41997-2ed6-428b-8ca3-d9541e8de149,<p>Explain about the 'Fuel management by Airli...,Easy,mcq_single_correct,original_question,e8a9b71a-8bdd-4208-a074-a3d62ab446c2,b05650e9-988e-4d43-ade1-5a0bad4489ae,6912740f-6977-4423-bfe1-ede3197c5796,VISTAS,SemExam,Unit 2,2020-12-10 08:41:12,77a55a51-0698-4d26-9396-08c05d689274,NaT,NaT,vistas296,[],{},ESODD2020_16CBAV54_DESC
8974587,af579e6a-f61b-4add-911c-b246658c117e,d27995ea-2ead-49c5-929f-af44191097e3,<p>Roles and responsibilities of an auditor in...,Easy,mcq_single_correct,original_question,e8a9b71a-8bdd-4208-a074-a3d62ab446c2,b05650e9-988e-4d43-ade1-5a0bad4489ae,8978d579-ea95-4811-b9c9-936f99315d4b,VISTAS,SemExam,Unit 5,2020-12-12 08:39:07,77a55a51-0698-4d26-9396-08c05d689274,NaT,NaT,vistas296,[],{},ESODD2020_15CPHDA5_DESC
8992765,e33ef61c-caad-409a-94a5-f4ebd4225190,8d2019b1-3ecc-427c-8466-b2236ffe9982,<p>Explain Fuelling of Aircraft?</p>,Easy,mcq_single_correct,original_question,e8a9b71a-8bdd-4208-a074-a3d62ab446c2,b05650e9-988e-4d43-ade1-5a0bad4489ae,c56e206c-4a5e-46b7-87ff-2fa53f7c6ab8,VISTAS,SemExam,Unit 1,2020-12-10 06:48:03,77a55a51-0698-4d26-9396-08c05d689274,NaT,NaT,vistas296,[],{},ESODD2020_16CBAV31_DESC
9001007,a934c494-1226-4068-8500-3dacc1a0ee82,f45257fa-73d1-413c-b18c-d1d185416409,<p>How Sea is polluted according to Edgar I Ba...,Easy,mcq_single_correct,original_question,e8a9b71a-8bdd-4208-a074-a3d62ab446c2,b05650e9-988e-4d43-ade1-5a0bad4489ae,6912740f-6977-4423-bfe1-ede3197c5796,VISTAS,SemExam,Unit 2,2020-12-10 10:15:17,77a55a51-0698-4d26-9396-08c05d689274,NaT,NaT,vistas296,[],{},ESODD2020_18LENG21_DESC- BCOM AF


In [44]:
# filtering the not deleted data from the whole data:
idx_create = set()
for i in range(len(mcq_single_correct_data["question_data"])):
    if mcq_single_correct_data["q_deletedAt"].iloc[i] is not pd.NaT:
        idx_create.add(i)
    elif mcq_single_correct_data["qb_deletedAt"].iloc[i] is not pd.NaT:
        idx_create.add(i)

In [45]:
len(list(idx_create))

795876

In [46]:
mcq_single_correct_data_notDeleted = mcq_single_correct_data.drop(mcq_single_correct_data.iloc[list(idx_create)].index)

In [47]:
'qb_id' in mcq_single_correct_data_notDeleted.columns

True

In [48]:
mcq_single_correct_data_notDeleted.shape

(7771636, 20)

In [49]:
# Empty question data rows
print(f"Number of questions in actual data :: {mcq_single_correct_data_notDeleted.shape[0]}")
no_ques_data = mcq_single_correct_data_notDeleted.iloc[np.where(mcq_single_correct_data_notDeleted['question_data'] == "<p></p>")]
print(f"Number of rows containing empty question :: {no_ques_data.shape[0]}")
no_ques_data.head(2)

Number of questions in actual data :: 7771636
Number of rows containing empty question :: 1024


,qb_id,q_id,question_data,manual_difficulty,question_type,imported,subject_id,topic_id,sub_topic_id,subject,topic,sub_topic,createdAt,school_id,q_deletedAt,qb_deletedAt,school_code,mcq_questions_options,mcq_questions_answer,qb_name
159535,8aee3d98-c276-4065-9b01-599c75cae58e,2faa8263-b96e-437e-84ef-46fa7b01cced,<p></p>,Medium,mcq_single_correct,original_question,None,None,None,None,None,None,2020-08-25 18:44:30,3e4ea61d-dca6-4ec8-94d9-f5f632878764,NaT,NaT,siet826,[],{},MPAT Hexaware 26.08.2020
162440,966b29e0-0ceb-4f1f-af59-02acf4f8a5e9,f33ccc65-f8d3-4203-aee2-f1f55c992009,<p></p>,Medium,mcq_single_correct,original_question,None,None,None,None,None,None,2020-10-16 05:18:00,3e4ea61d-dca6-4ec8-94d9-f5f632878764,NaT,NaT,siet826,[],{},CTS Topic-Wise QP 16.10.2020


In [50]:
cleaned_data1 = mcq_single_correct_data_notDeleted.drop(index=no_ques_data.index,axis=0)

In [51]:
# removing the html tags from the data
cleaned_data2 = duplicate.clean_data(cleaned_data1)

Data cleaned...
Time taken :: 32.609764099121094


In [52]:
# remove the nan created while removing the html tags
cleaned_data3 = duplicate.clean_nan(cleaned_data2)
cleaned_data3.shape

(7770612, 21)

In [53]:
#inserting column to the data:
duplicate.insert_col(cleaned_data3,18,"cleaned_mcq_questions_options",value="")

In [54]:
cleaned_data3.head()

,qb_id,q_id,question_data,clean_question_data,manual_difficulty,question_type,imported,subject_id,topic_id,sub_topic_id,...,sub_topic,createdAt,school_id,q_deletedAt,qb_deletedAt,school_code,cleaned_mcq_questions_options,mcq_questions_options,mcq_questions_answer,qb_name
0,c3e6cf7c-393d-4e51-b2da-c5fa81efaa8f,c47ef293-5193-4910-a92a-8195509ddb90,"<p><span style=""color: rgb(0, 0, 10);"">Digital...",Digital Computers use ___________ system.,Easy,mcq_single_correct,original_question,6ed7b6f1-be79-494b-b488-bbc4a2156f95,43de6b0e-c7fe-45ca-9e42-f5cc9120b15f,95641ed5-1986-401a-b3b9-a21ac54a3f38,...,CO1,2021-10-25 20:04:59,c7bde760-cc4a-4d2d-a91d-0fe6031a8ff1,NaT,NaT,psgtech850,,"[{""text"":""<p><span style=\""color: rgb(0, 0, 10...","{""args"":[""<p><span style=\""color: rgb(0, 0, 10...",BACKUP_QB_15I302/19I302 COMPUTER ARCHITECTURE_...
1,530f79d2-fa66-4ef7-9ea1-e083a6504946,a8257d80-a435-45b9-a67a-a11aace7147e,"<p><span style=""color: rgb(0, 0, 10);"">_______...",_______________ are defined as psychosocial ab...,Easy,mcq_single_correct,original_question,c1ecede5-ce85-404a-b086-211fcf0de76e,2fc15da6-d65b-4055-b022-ac262bc9486a,0c911d14-73dc-4043-95c0-c82566e245cf,...,CO5,2021-03-11 05:57:51,bc65162a-72ff-496a-bd14-7f23335ea6f0,NaT,NaT,rmkec496,,"[{""text"":""<p><span style=\""color: rgb(0, 0, 10...","{""args"":[""<p><span style=\""color: rgb(0, 0, 10...",20EL102_BCVS1_CB_ES_AT_PartB
2,be4f0063-bcb0-4899-a94f-628727a77417,add83be4-c92e-40e6-a947-b7e5f5ced412,"<p><span style=""color: rgb(0, 0, 10);"">T</span...",The planning done by top management is-------...,Hard,mcq_single_correct,original_question,6ed7b6f1-be79-494b-b488-bbc4a2156f95,43de6b0e-c7fe-45ca-9e42-f5cc9120b15f,95641ed5-1986-401a-b3b9-a21ac54a3f38,...,CO1,2021-10-30 06:58:01,c7bde760-cc4a-4d2d-a91d-0fe6031a8ff1,NaT,NaT,psgtech850,,"[{""text"":""<p><span style=\""color: rgb(0, 0, 10...","{""args"":[""<p><span style=\""color: rgb(0, 0, 10...",QB_18FD53 APPAREL PRODUCTION PLANNING AND CONT...
3,3da85cdd-3a7d-4d71-870f-a18cf8a207e7,adb3f306-9069-488c-802d-2da195ec20c3,"<p>Assume double[][] x = new double[4][5], wha...","Assume double[][] x = new double[4][5], what a...",Medium,mcq_single_correct,unlimited_purchase,9f91343b-be0e-4336-b199-30d823a58046,bdbf4244-4656-4c89-a7e7-4ed44b06d8fb,4d31960b-1a52-41d3-83f6-13879e900e2b,...,Multidimensional array,2020-07-25 09:31:38,1e219353-c3bf-48b0-9825-e17ff94ec4f2,NaT,NaT,abes819,,"[{""text"":""<p>4 and 4</p>"",""media"":""""},{""text"":...","{""args"":[""<p>4 and 5</p>""],""partial"":[]}",HoP_Technical Test 1
5,6494f54a-5c51-4a96-b362-4c537bdbdd67,b777a2ec-3375-44a6-998f-cba268626992,<p>Choose the correct option from given below:...,Choose the correct option from given below:,Medium,mcq_single_correct,unlimited_purchase,15277909-f542-478d-9c4b-3de8e7812734,fe7247db-8d74-4a5f-9dd5-f821d7a451b5,9f6a191b-1057-4259-aee0-11595fcf245e,...,UML,2021-07-27 14:25:33,eabcfa6a-7f7c-4457-95e9-33ee2dd1367e,NaT,NaT,mcet848,,"[{""text"":""<p>Prototyping Model facilitates reu...","{""args"":[""<p>Both RAD &amp; Prototyping Model ...",L and T _ Round 1 _Test 10


In [55]:
# transforming the cleaned data

transformed_data = duplicate.transform_data(cleaned_data3)
transformed_data.shape

(7770612, 22)

In [56]:
filtered_data = duplicate.filter_duplicate(transformed_data)
filtered_data.shape

(6210031, 24)

In [57]:
# collecting the index of duplicates of a random single question
num = 70
idx = duplicate.find_dup_idx(filtered_data,cleaned_data3["clean_question_data"].iloc[num])
print(f"""Number of duplicate for question :: {cleaned_data3["clean_question_data"].iloc[num]}\n is {len(idx)}""")

Number of duplicate for question :: choose the correct option from given below:
 is 42


In [58]:
# fetching the variations of the above question
duplicate.variations(filtered_data=filtered_data,idx=idx)

filtered_data :: 42
q_type :: 42
q_sub :: 42
q_topic :: 42
q_sub_topic :: 42
Variation found...
Time taken :: 0.01563739776611328


,index,qb_id,q_id,question_data,clean_question_data,manual_difficulty,question_type,imported,subject_id,topic_id,...,createdAt,school_id,q_deletedAt,qb_deletedAt,school_code,cleaned_mcq_questions_options,mcq_questions_options,mcq_questions_answer,qb_name,duplicate
0,95,ca091f31-5247-4252-8962-8f6832cb693c,e2f67655-a008-462b-8ca1-4b036873200c,<p>Choose the correct option from given below:...,choose the correct option from given below:,Medium,mcq_single_correct,is_imported_question,15277909-f542-478d-9c4b-3de8e7812734,fe7247db-8d74-4a5f-9dd5-f821d7a451b5,...,2020-02-13 04:04:37,726462db-6397-4906-8793-df0406b5133a,NaT,NaT,neohire,,"[{""text"":""<p>Prototyping Model facilitates reu...","{""args"":[""<p>Both RAD &amp; Prototyping Model ...",SE UML_SDLC_MCQ,True
3629,18312,b7155bc3-3310-4ed9-9d13-a13a71e9adb0,11ce042f-d2a7-4f10-ac0c-5e1faba17ebd,<p>Choose the correct option from given below:...,choose the correct option from given below:,Medium,mcq_single_correct,is_imported_question,15277909-f542-478d-9c4b-3de8e7812734,fe7247db-8d74-4a5f-9dd5-f821d7a451b5,...,2020-04-26 17:51:00,e236d981-b79d-4910-8d05-861970e24bc8,NaT,NaT,rmkinstitutions571,,"[{""text"":""<p>Prototyping Model facilitates reu...","{""args"":[""<p>Both RAD &amp; Prototyping Model ...",FP_PT_SDLC,True
10606,39389,68858a65-cbf3-4fb5-99c1-3e7c1af083f3,010f12f0-7161-4e0f-95f4-69ee2ab09a68,<p>Choose the correct option from given below:...,choose the correct option from given below:,Medium,mcq_single_correct,unlimited_purchase,15277909-f542-478d-9c4b-3de8e7812734,fe7247db-8d74-4a5f-9dd5-f821d7a451b5,...,2021-07-24 13:22:42,96225671-55b9-45e8-a632-59da09d02740,NaT,NaT,kctplacement196,,"[{""text"":""<p>Prototyping Model facilitates reu...","{""args"":[""<p>Both RAD &amp; Prototyping Model ...",L and T _ Round 1 _Test 10,True
17359,57688,9d531079-6114-4cd5-9685-1a216dc6cac5,5bc5c80d-d1f3-4252-b6d7-5c5942358a23,<p>Choose the correct option from given below:...,choose the correct option from given below:,Medium,mcq_single_correct,unlimited_purchase,15277909-f542-478d-9c4b-3de8e7812734,fe7247db-8d74-4a5f-9dd5-f821d7a451b5,...,2021-08-03 09:45:52,8399f359-f86b-4bc7-9d88-85886446788f,NaT,NaT,karunya711,,"[{""text"":""<p>Prototyping Model facilitates reu...","{""args"":[""<p>Both RAD &amp; Prototyping Model ...",L and T _ Round 1 _Test 10,True
18521,60594,def62cda-c5fb-44ef-a8db-6f93a0ce4923,866a7749-c1e7-4d79-88aa-1d77d76bda2c,<p>Choose the correct option from given below:...,choose the correct option from given below:,Medium,mcq_single_correct,unlimited_purchase,15277909-f542-478d-9c4b-3de8e7812734,fe7247db-8d74-4a5f-9dd5-f821d7a451b5,...,2021-07-29 07:29:31,9bc28106-1a66-49b7-92be-07862e083809,NaT,NaT,cbit450,,"[{""text"":""<p>Prototyping Model facilitates reu...","{""args"":[""<p>Both RAD &amp; Prototyping Model ...",L and T _ Round 1 _Test 10,True
25940,79532,2533fa8a-a8c1-4b53-b429-415050d273a0,0cf5492d-5e20-4008-8ffa-c924d66ece3c,<p>Choose the correct option from given below:...,choose the correct option from given below:,Medium,mcq_single_correct,unlimited_purchase,15277909-f542-478d-9c4b-3de8e7812734,fe7247db-8d74-4a5f-9dd5-f821d7a451b5,...,2021-07-29 07:16:16,8248c077-4fab-417d-84dc-41e5a29adfa3,NaT,NaT,kcg168,,"[{""text"":""<p>Prototyping Model facilitates reu...","{""args"":[""<p>Both RAD &amp; Prototyping Model ...",L and T _ Round 1 _Test 10,True
26864,81800,ca1f18e4-ddd6-4672-9963-0f633c39be3e,fdb439fb-7398-4247-b853-a3c7f0cceff1,<p>Choose the correct option from given below:...,choose the correct option from given below:,Medium,mcq_single_correct,unlimited_purchase,15277909-f542-478d-9c4b-3de8e7812734,fe7247db-8d74-4a5f-9dd5-f821d7a451b5,...,2021-07-28 14:39:47,26667212-4bbb-427f-9bf4-10b5d794c30b,NaT,NaT,sathyabama250,,"[{""text"":""<p>Prototyping Model facilitates reu...","{""args"":[""<p>Both RAD &amp; Prototyping Model ...",L and T _ Round 1 _Test 10,True
62656,167976,c972e79c-f27d-40c9-8b4e-0b6c10b401ea,99389541-ed81-4ce8-80fe-52048fc698ee,<p>Choose the corre

In [59]:
# duplicated data:
dup = duplicate.variations(filtered_data=filtered_data,idx=idx)

filtered_data :: 42
q_type :: 42
q_sub :: 42
q_topic :: 42
q_sub_topic :: 42
Variation found...
Time taken :: 0.0016446113586425781


In [60]:
dup.head()

,index,qb_id,q_id,question_data,clean_question_data,manual_difficulty,question_type,imported,subject_id,topic_id,...,createdAt,school_id,q_deletedAt,qb_deletedAt,school_code,cleaned_mcq_questions_options,mcq_questions_options,mcq_questions_answer,qb_name,duplicate
0,95,ca091f31-5247-4252-8962-8f6832cb693c,e2f67655-a008-462b-8ca1-4b036873200c,<p>Choose the correct option from given below:...,choose the correct option from given below:,Medium,mcq_single_correct,is_imported_question,15277909-f542-478d-9c4b-3de8e7812734,fe7247db-8d74-4a5f-9dd5-f821d7a451b5,...,2020-02-13 04:04:37,726462db-6397-4906-8793-df0406b5133a,NaT,NaT,neohire,,"[{""text"":""<p>Prototyping Model facilitates reu...","{""args"":[""<p>Both RAD &amp; Prototyping Model ...",SE UML_SDLC_MCQ,True
3629,18312,b7155bc3-3310-4ed9-9d13-a13a71e9adb0,11ce042f-d2a7-4f10-ac0c-5e1faba17ebd,<p>Choose the correct option from given below:...,choose the correct option from given below:,Medium,mcq_single_correct,is_imported_question,15277909-f542-478d-9c4b-3de8e7812734,fe7247db-8d74-4a5f-9dd5-f821d7a451b5,...,2020-04-26 17:51:00,e236d981-b79d-4910-8d05-861970e24bc8,NaT,NaT,rmkinstitutions571,,"[{""text"":""<p>Prototyping Model facilitates reu...","{""args"":[""<p>Both RAD &amp; Prototyping Model ...",FP_PT_SDLC,True
10606,39389,68858a65-cbf3-4fb5-99c1-3e7c1af083f3,010f12f0-7161-4e0f-95f4-69ee2ab09a68,<p>Choose the correct option from given below:...,choose the correct option from given below:,Medium,mcq_single_correct,unlimited_purchase,15277909-f542-478d-9c4b-3de8e7812734,fe7247db-8d74-4a5f-9dd5-f821d7a451b5,...,2021-07-24 13:22:42,96225671-55b9-45e8-a632-59da09d02740,NaT,NaT,kctplacement196,,"[{""text"":""<p>Prototyping Model facilitates reu...","{""args"":[""<p>Both RAD &amp; Prototyping Model ...",L and T _ Round 1 _Test 10,True
17359,57688,9d531079-6114-4cd5-9685-1a216dc6cac5,5bc5c80d-d1f3-4252-b6d7-5c5942358a23,<p>Choose the correct option from given below:...,choose the correct option from given below:,Medium,mcq_single_correct,unlimited_purchase,15277909-f542-478d-9c4b-3de8e7812734,fe7247db-8d74-4a5f-9dd5-f821d7a451b5,...,2021-08-03 09:45:52,8399f359-f86b-4bc7-9d88-85886446788f,NaT,NaT,karunya711,,"[{""text"":""<p>Prototyping Model facilitates reu...","{""args"":[""<p>Both RAD &amp; Prototyping Model ...",L and T _ Round 1 _Test 10,True
18521,60594,def62cda-c5fb-44ef-a8db-6f93a0ce4923,866a7749-c1e7-4d79-88aa-1d77d76bda2c,<p>Choose the correct option from given below:...,choose the correct option from given below:,Medium,mcq_single_correct,unlimited_purchase,15277909-f542-478d-9c4b-3de8e7812734,fe7247db-8d74-4a5f-9dd5-f821d7a451b5,...,2021-07-29 07:29:31,9bc28106-1a66-49b7-92be-07862e083809,NaT,NaT,cbit450,,"[{""text"":""<p>Prototyping Model facilitates reu...","{""args"":[""<p>Both RAD &amp; Prototyping Model ...",L and T _ Round 1 _Test 10,True


In [61]:
# cleaning the data answer data:
cleaned_data3 = duplicate.clean_answer_data(dup)

Column present....
0
Answer Data cleaned...
Time taken :: 0.027760744094848633


/opt/conda/lib/python3.7/site-packages/pandas/core/indexing.py:1732: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_single_block(indexer, value, name)


In [62]:
cleaned_data3

,index,qb_id,q_id,question_data,clean_question_data,manual_difficulty,question_type,imported,subject_id,topic_id,...,createdAt,school_id,q_deletedAt,qb_deletedAt,school_code,cleaned_mcq_questions_options,mcq_questions_options,mcq_questions_answer,qb_name,duplicate
0,95,ca091f31-5247-4252-8962-8f6832cb693c,e2f67655-a008-462b-8ca1-4b036873200c,<p>Choose the correct option from given below:...,choose the correct option from given below:,Medium,mcq_single_correct,is_imported_question,15277909-f542-478d-9c4b-3de8e7812734,fe7247db-8d74-4a5f-9dd5-f821d7a451b5,...,2020-02-13 04:04:37,726462db-6397-4906-8793-df0406b5133a,NaT,NaT,neohire,[Prototyping Model facilitates reusability of ...,"[{""text"":""<p>Prototyping Model facilitates reu...","{""args"":[""<p>Both RAD &amp; Prototyping Model ...",SE UML_SDLC_MCQ,True
3629,18312,b7155bc3-3310-4ed9-9d13-a13a71e9adb0,11ce042f-d2a7-4f10-ac0c-5e1faba17ebd,<p>Choose the correct option from given below:...,choose the correct option from given below:,Medium,mcq_single_correct,is_imported_question,15277909-f542-478d-9c4b-3de8e7812734,fe7247db-8d74-4a5f-9dd5-f821d7a451b5,...,2020-04-26 17:51:00,e236d981-b79d-4910-8d05-861970e24bc8,NaT,NaT,rmkinstitutions571,[Prototyping Model facilitates reusability of ...,"[{""text"":""<p>Prototyping Model facilitates reu...","{""args"":[""<p>Both RAD &amp; Prototyping Model ...",FP_PT_SDLC,True
10606,39389,68858a65-cbf3-4fb5-99c1-3e7c1af083f3,010f12f0-7161-4e0f-95f4-69ee2ab09a68,<p>Choose the correct option from given below:...,choose the correct option from given below:,Medium,mcq_single_correct,unlimited_purchase,15277909-f542-478d-9c4b-3de8e7812734,fe7247db-8d74-4a5f-9dd5-f821d7a451b5,...,2021-07-24 13:22:42,96225671-55b9-45e8-a632-59da09d02740,NaT,NaT,kctplacement196,[Prototyping Model facilitates reusability of ...,"[{""text"":""<p>Prototyping Model facilitates reu...","{""args"":[""<p>Both RAD &amp; Prototyping Model ...",L and T _ Round 1 _Test 10,True
17359,57688,9d531079-6114-4cd5-9685-1a216dc6cac5,5bc5c80d-d1f3-4252-b6d7-5c5942358a23,<p>Choose the correct option from given below:...,choose the correct option from given below:,Medium,mcq_single_correct,unlimited_purchase,15277909-f542-478d-9c4b-3de8e7812734,fe7247db-8d74-4a5f-9dd5-f821d7a451b5,...,2021-08-03 09:45:52,8399f359-f86b-4bc7-9d88-85886446788f,NaT,NaT,karunya711,[Prototyping Model facilitates reusability of ...,"[{""text"":""<p>Prototyping Model facilitates reu...","{""args"":[""<p>Both RAD &amp; Prototyping Model ...",L and T _ Round 1 _Test 10,True
18521,60594,def62cda-c5fb-44ef-a8db-6f93a0ce4923,866a7749-c1e7-4d79-88aa-1d77d76bda2c,<p>Choose the correct option from given below:...,choose the correct option from given below:,Medium,mcq_single_correct,unlimited_purchase,15277909-f542-478d-9c4b-3de8e7812734,fe7247db-8d74-4a5f-9dd5-f821d7a451b5,...,2021-07-29 07:29:31,9bc28106-1a66-49b7-92be-07862e083809,NaT,NaT,cbit450,[Prototyping Model facilitates reusability of ...,"[{""text"":""<p>Prototyping Model facilitates reu...","{""args"":[""<p>Both RAD &amp; Prototyping Model ...",L and T _ Round 1 _Test 10,True
25940,79532,2533fa8a-a8c1-4b53-b429-415050d273a0,0cf5492d-5e20-4008-8ffa-c924d66ece3c,<p>Choose the correct option from given below:...,choose the correct option from given below:,Medium,mcq_single_correct,unlimited_purchase,15277909-f542-478d-9c4b-3de8e7812734,fe7247db-8d74-4a5f-9dd5-f821d7a451b5,...,2021-07-29 07:16:16,8248c077-4fab-417d-84dc-41e5a29adfa3,NaT,NaT,kcg168,[Prototyping Model facilitates reusability of ...,"[{""text"":""<p>Prototyping Model facilitates reu...","{""args"":[""<p>Both RAD &amp; Prototyping Model ...",L and T _ Round 1 _Test 10,True
26864,81800,ca1f18e4-ddd6-4672-9963-0f633c39be3e,fdb439fb-7398-4247-b853-a3c7f0cceff1,<p>Choose the correct option from given below:...,choose the correct option from given below:,Medium,mcq_single_correct,unlimited_purchase,15277909-f542-478d-9c4b-3de8e7812734,fe7247db-8d74-4a5f-9dd5-f821d7a451b5,...,2021-07-28 14:39:47,26667212-4bbb-

In [66]:
cleaned_data3["cleaned_mcq_questions_options"].duplicated()

0          False
3629        True
10606       True
17359       True
18521       True
25940       True
26864       True
62656       True
64516       True
83086       True
84266       True
106367      True
116096      True
136232      True
161767      True
166756      True
171306      True
172222      True
181948     False
205490      True
216670      True
218805      True
224138      True
232825      True
234831     False
255996      True
281334      True
281951      True
292382      True
304563      True
310784      True
322814      True
358721      True
359448      True
391737      True
403497      True
455939      True
641752      True
725957      True
909791     False
1040984    False
1114470    False
Name: cleaned_mcq_questions_options, dtype: bool

In [74]:
cleaned_data3["cleaned_mcq_questions_options"].iloc[np.where(cleaned_data3["cleaned_mcq_questions_options"].duplicated() == False)].iloc[1]

['Prototyping Model facilitates reusability of components',
 'RAD Model Model facilitates reusability of components',
 'Both RAD  Prototyping Model facilitates reusability of components',
 ' None of these']

In [72]:
cleaned_data3["cleaned_mcq_questions_options"].iloc[np.where(cleaned_data3["cleaned_mcq_questions_options"].duplicated() == True)].iloc[2]

['Prototyping Model facilitates reusability of components',
 'RAD Model Model facilitates reusability of components',
 'Both RAD  Prototyping Model facilitates reusability of components',
 'None']

# Introducing the whole data to the algo:

In [6]:
whole_data.head()

,qb_id,q_id,question_data,manual_difficulty,question_type,imported,subject_id,topic_id,sub_topic_id,subject,topic,sub_topic,createdAt,school_id,q_deletedAt,qb_deletedAt,school_code,mcq_questions_options,mcq_questions_answer,qb_name
0,1705eb77-3669-404e-9c09-4c1228b03518,7d9aec3a-16fc-4b4b-838b-34b0b47f9df0,"<p>In a fixed time, a boy swims double the dis...",Hard,mcq_single_correct,unlimited_purchase,3639c0f6-9b40-43d2-aafc-90150e1cbf7b,7b5e555e-5199-4955-8462-292c6a975daa,772ddc31-6c49-4796-b765-6594dfeb651c,Aptitude,Quantitative Ability,Boats and Streams,2020-10-13 05:51:58,de2101bc-0739-40ce-b5aa-91583d5ac0c2,NaT,NaT,jeppiaar645,"[{""text"":""<p>7 km/hr</p>"",""media"":""""},{""text"":...","{""args"":[""<p>9 km/hr</p>""],""partial"":[]}",Cognizant New Test 4
1,1cac6231-b1d3-446c-81d4-2e4f9166bfc0,b7951331-fe7f-4c50-a75c-9ee4c558faba,"<p><span style=""color: rgb(0, 0, 10);"">Managem...",Easy,mcq_single_correct,original_question,c1ecede5-ce85-404a-b086-211fcf0de76e,2fc15da6-d65b-4055-b022-ac262bc9486a,0c911d14-73dc-4043-95c0-c82566e245cf,End Semester,Unit 5,CO5,2020-12-31 10:36:28,bc65162a-72ff-496a-bd14-7f23335ea6f0,NaT,NaT,rmkec496,"[{""text"":""<p><span style=\""color: rgb(0, 0, 10...","{""args"":[""<p><span style=\""color: rgb(0, 0, 10...","MG8591_PRINCIPLES OF MANAGEMENT_CSE ,IT _ES_AT..."
2,721f4853-11bc-40a2-8a6a-b0b52ae9ddab,9a257114-b604-4ec7-84b5-9a29e9d36cac,<p>Arun is interested in a new function such t...,Medium,programming,original_question,15277909-f542-478d-9c4b-3de8e7812734,dde0776e-67c4-4f29-8296-9698c4114ed0,509aa957-ac23-4b5c-9416-0bcce280b42d,Programming,Algorithms,Mathematical Algorithms,2020-04-25 07:03:33,69d62be1-d6bb-4216-b8bf-4372b36f4942,NaT,NaT,placementseason668,None,None,IT Service_Gobinath_April
3,0319fee4-5d54-4ab8-8575-b11061ce2637,994a97a8-50f0-4586-92ef-1531265c63e9,<p>An amplifier has a gain of 60 dB with a Ban...,Easy,mcq_single_correct,unlimited_purchase,5e2f4532-bf80-4a29-8599-16f91a09d478,5ae2a04e-e443-4981-aa79-584ff775ed05,52dc14d8-246a-4272-be90-08dbd5fcd042,Engineering,Electronics,Analog Electronics,2020-08-21 12:55:21,5ebeaefb-df6d-4d91-bb5f-f7062e351ed5,NaT,NaT,vnrvjiet858,"[{""text"":""<p><span style=\""color: rgb(0, 0, 0)...","{""args"":[""<p><span style=\""color: rgb(0, 0, 0)...",Hexaware Assessment 1_ECE/EEE
4,269b549a-4a45-4f4a-a211-1f237dda4c2c,7bb5db30-1dd9-4280-9c53-748eef00f86c,"<p><span style=""color: rgb(0, 0, 10);"">Which <...",Easy,mcq_single_correct,original_question,9023b46a-5c87-45eb-bd49-df7568728f23,d352af2b-dd86-4e2a-92bd-7faab08946ae,b818c93d-342e-4039-a0c0-89ee2d73f363,Kongu,End Semester,Unit_3,2021-08-14 06:59:33,384b10f0-cfed-4921-8db5-c2bdeeb51344,NaT,NaT,kongu426,"[{""text"":""<p><span style=\""color: rgb(0, 0, 10...","{""args"":[""<p><span style=\""color: rgb(0, 0, 10...",14FTT34_FB_FT_ES__AUG_2021


In [7]:
# empty answer rows whose question deletion or qb deletion data is absent.

idx = [] 
for i in range(len(whole_data["question_data"])):
    if whole_data["q_deletedAt"].iloc[i] is pd.NaT:
        if whole_data["qb_deletedAt"].iloc[i] is pd.NaT:
            if whole_data["mcq_questions_options"].iloc[i] == '[]':
                idx.append(i)

In [8]:
len(idx)

2330

In [9]:
whole_data.iloc[idx]

,qb_id,q_id,question_data,manual_difficulty,question_type,imported,subject_id,topic_id,sub_topic_id,subject,topic,sub_topic,createdAt,school_id,q_deletedAt,qb_deletedAt,school_code,mcq_questions_options,mcq_questions_answer,qb_name
279862,b400f0e1-d4dd-40d5-851e-14a90b9dae02,0e6e65a2-138f-49a4-8bf2-f8cb49a4ab2c,<p>Choose the correct Preposition: I am very b...,Easy,mcq_multiple_correct,original_question,3639c0f6-9b40-43d2-aafc-90150e1cbf7b,1c73a995-728c-4e2a-b9bb-262db072f58a,8a9c15bc-1fc4-4e3e-908a-80bad788c100,Aptitude,Verbal Ability,Prepositions,2022-03-09 07:22:35,6535d3dc-bbc6-4efc-bba3-6f0a2db5bea7,NaT,NaT,nandha552,[],"{""args"":[],""partial"":[],""reductionPercent"":0}",GRAMMAR
288443,419cd94b-ad18-4580-9dc0-b7c4bf825e11,18befd39-f8fb-479c-8365-b98044e3588e,<p>return 0; }</p>,Medium,mcq_single_correct,original_question,fc9d8672-4f7a-4800-8f05-294dc460feda,b7425c21-2670-4bcd-ada4-24847c8c9408,55021f2e-e729-4ade-99d1-f8d14879e041,bulk-imported,bulk-imported,bulk-imported,2018-08-30 14:22:38,6ff9556e-0409-48a2-b834-9f740f56a5e6,NaT,NaT,talentio627,[],{},Capgemini QB 4 - technical
424602,40d0c765-da46-4b7e-9b38-391fd1e70150,099f94b3-79c5-468f-84fb-b557227b5ab6,<p>undefined</p>,Medium,mcq_single_correct,original_question,None,None,None,None,None,None,2020-12-11 15:51:04,77a55a51-0698-4d26-9396-08c05d689274,NaT,NaT,vistas296,[],{},TEST_TEST_TEST
424611,966b29e0-0ceb-4f1f-af59-02acf4f8a5e9,6cbd4895-9078-415d-936a-786c6a7e982c,<p></p>,Medium,mcq_single_correct,original_question,None,None,None,None,None,None,2020-10-16 05:18:04,3e4ea61d-dca6-4ec8-94d9-f5f632878764,NaT,NaT,siet826,[],{},CTS Topic-Wise QP 16.10.2020
435583,40d0c765-da46-4b7e-9b38-391fd1e70150,82a53364-28de-4fc7-85b4-371a99df650b,<p>undefined</p>,Medium,mcq_single_correct,original_question,None,None,None,None,None,None,2020-12-11 15:50:29,77a55a51-0698-4d26-9396-08c05d689274,NaT,NaT,vistas296,[],{},TEST_TEST_TEST
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9175268,1604127b-b4c7-4247-8e79-070ec8d3d0be,a171595b-2992-40ba-9450-d52036612b26,<p>Write the structure and functions of glucos...,Easy,mcq_single_correct,original_question,e8a9b71a-8bdd-4208-a074-a3d62ab446c2,b05650e9-988e-4d43-ade1-5a0bad4489ae,c56e206c-4a5e-46b7-87ff-2fa53f7c6ab8,VISTAS,SemExam,Unit 1,2020-12-10 10:53:50,77a55a51-0698-4d26-9396-08c05d689274,NaT,NaT,vistas296,[],{},ESODD2020_12CBBC31_DESC
9182132,e33ef61c-caad-409a-94a5-f4ebd4225190,b54d4d55-5662-4db4-8260-901ed73fd36a,<p>Explain Categories of ILS?</p>,Easy,mcq_single_correct,original_question,e8a9b71a-8bdd-4208-a074-a3d62ab446c2,b05650e9-988e-4d43-ade1-5a0bad4489ae,6912740f-6977-4423-bfe1-ede3197c5796,VISTAS,SemExam,Unit 2,2020-12-10 06:48:03,77a55a51-0698-4d26-9396-08c05d689274,NaT,NaT,vistas296,[],{},ESODD2020_16CBAV31_DESC
9192037,a934c494-1226-4068-8500-3dacc1a0ee82,c210d706-2194-473f-a826-c10317f1b893,"<p>What are the plans of Soapy, a homeless tra...",Easy,mcq_single_correct,original_question,e8a9b71a-8bdd-4208-a074-a3d62ab446c2,b05650e9-988e-4d43-ade1-5a0bad4489ae,03a71094-137d-4987-8d96-3ecd10a21346,VISTAS,SemExam,Unit 4,2020-12-10 10:15:18,77a55a51-0698-4d26-9396-08c05d689274,NaT,NaT,vistas296,[],{},ESODD2020_18LENG21_DESC- BCOM AF
9202737,1604127b-b4c7-4247-8e79-070ec8d3d0be,ec0bb998-89b3-4d88-a2a0-1389b04e6b04,<p>Discuss the various bonds that play an impo...,Easy,mcq_single_correct,original_question,e8a9b71a-8bdd-4208-a074-a3d62ab446c2,b05650e9-988e-4d43-ade1-5a0bad4489ae,03a71094-137d-4987-8d96-3ecd10a21346,VISTAS,SemExam,Unit 4,2020-12-10 10:53:51,77a55a51-0698-4d26-9396-08c05d689274,NaT,NaT,vistas296,[],{},ESODD2020_12CBBC31_DESC


In [10]:
# filtering the not deleted data from the whole data:
idx_create = set()
for i in range(len(whole_data["question_data"])):
    if whole_data["q_deletedAt"].iloc[i] is not pd.NaT:
        idx_create.add(i)
    elif whole_data["qb_deletedAt"].iloc[i] is not pd.NaT:
        idx_create.add(i)

In [11]:
# total amount of data that has a record of deletion in whole data
len(idx_create)

838869

In [12]:
# Separating the not deleted data from whole data:
whole_data_notDeleted = whole_data.drop(whole_data.iloc[list(idx_create)].index)

In [13]:
# Actual data hving all data that are not deleted from the front_end:
whole_data_notDeleted.shape

(8440235, 20)

In [14]:
whole_data_notDeleted.head()

,qb_id,q_id,question_data,manual_difficulty,question_type,imported,subject_id,topic_id,sub_topic_id,subject,topic,sub_topic,createdAt,school_id,q_deletedAt,qb_deletedAt,school_code,mcq_questions_options,mcq_questions_answer,qb_name
0,1705eb77-3669-404e-9c09-4c1228b03518,7d9aec3a-16fc-4b4b-838b-34b0b47f9df0,"<p>In a fixed time, a boy swims double the dis...",Hard,mcq_single_correct,unlimited_purchase,3639c0f6-9b40-43d2-aafc-90150e1cbf7b,7b5e555e-5199-4955-8462-292c6a975daa,772ddc31-6c49-4796-b765-6594dfeb651c,Aptitude,Quantitative Ability,Boats and Streams,2020-10-13 05:51:58,de2101bc-0739-40ce-b5aa-91583d5ac0c2,NaT,NaT,jeppiaar645,"[{""text"":""<p>7 km/hr</p>"",""media"":""""},{""text"":...","{""args"":[""<p>9 km/hr</p>""],""partial"":[]}",Cognizant New Test 4
1,1cac6231-b1d3-446c-81d4-2e4f9166bfc0,b7951331-fe7f-4c50-a75c-9ee4c558faba,"<p><span style=""color: rgb(0, 0, 10);"">Managem...",Easy,mcq_single_correct,original_question,c1ecede5-ce85-404a-b086-211fcf0de76e,2fc15da6-d65b-4055-b022-ac262bc9486a,0c911d14-73dc-4043-95c0-c82566e245cf,End Semester,Unit 5,CO5,2020-12-31 10:36:28,bc65162a-72ff-496a-bd14-7f23335ea6f0,NaT,NaT,rmkec496,"[{""text"":""<p><span style=\""color: rgb(0, 0, 10...","{""args"":[""<p><span style=\""color: rgb(0, 0, 10...","MG8591_PRINCIPLES OF MANAGEMENT_CSE ,IT _ES_AT..."
2,721f4853-11bc-40a2-8a6a-b0b52ae9ddab,9a257114-b604-4ec7-84b5-9a29e9d36cac,<p>Arun is interested in a new function such t...,Medium,programming,original_question,15277909-f542-478d-9c4b-3de8e7812734,dde0776e-67c4-4f29-8296-9698c4114ed0,509aa957-ac23-4b5c-9416-0bcce280b42d,Programming,Algorithms,Mathematical Algorithms,2020-04-25 07:03:33,69d62be1-d6bb-4216-b8bf-4372b36f4942,NaT,NaT,placementseason668,None,None,IT Service_Gobinath_April
3,0319fee4-5d54-4ab8-8575-b11061ce2637,994a97a8-50f0-4586-92ef-1531265c63e9,<p>An amplifier has a gain of 60 dB with a Ban...,Easy,mcq_single_correct,unlimited_purchase,5e2f4532-bf80-4a29-8599-16f91a09d478,5ae2a04e-e443-4981-aa79-584ff775ed05,52dc14d8-246a-4272-be90-08dbd5fcd042,Engineering,Electronics,Analog Electronics,2020-08-21 12:55:21,5ebeaefb-df6d-4d91-bb5f-f7062e351ed5,NaT,NaT,vnrvjiet858,"[{""text"":""<p><span style=\""color: rgb(0, 0, 0)...","{""args"":[""<p><span style=\""color: rgb(0, 0, 0)...",Hexaware Assessment 1_ECE/EEE
4,269b549a-4a45-4f4a-a211-1f237dda4c2c,7bb5db30-1dd9-4280-9c53-748eef00f86c,"<p><span style=""color: rgb(0, 0, 10);"">Which <...",Easy,mcq_single_correct,original_question,9023b46a-5c87-45eb-bd49-df7568728f23,d352af2b-dd86-4e2a-92bd-7faab08946ae,b818c93d-342e-4039-a0c0-89ee2d73f363,Kongu,End Semester,Unit_3,2021-08-14 06:59:33,384b10f0-cfed-4921-8db5-c2bdeeb51344,NaT,NaT,kongu426,"[{""text"":""<p><span style=\""color: rgb(0, 0, 10...","{""args"":[""<p><span style=\""color: rgb(0, 0, 10...",14FTT34_FB_FT_ES__AUG_2021


# Idea to find the duplicates:

1. We will first find the duplicates from the data that are present im both back-end and front-end.

2. If required, we will find the duplicates for the data that are deleted from the front-end

In [15]:
# Finding Empty question data rows:

print(f"Number of questions in actual data :: {whole_data_notDeleted.shape[0]}")
no_ques_data = whole_data_notDeleted.iloc[np.where(whole_data_notDeleted['question_data'] == "<p></p>")]
print(f"Number of rows containing empty question :: {no_ques_data.shape[0]}")
no_ques_data.head(2)

Number of questions in actual data :: 8440235
Number of rows containing empty question :: 1024


,qb_id,q_id,question_data,manual_difficulty,question_type,imported,subject_id,topic_id,sub_topic_id,subject,topic,sub_topic,createdAt,school_id,q_deletedAt,qb_deletedAt,school_code,mcq_questions_options,mcq_questions_answer,qb_name
424611,966b29e0-0ceb-4f1f-af59-02acf4f8a5e9,6cbd4895-9078-415d-936a-786c6a7e982c,<p></p>,Medium,mcq_single_correct,original_question,None,None,None,None,None,None,2020-10-16 05:18:04,3e4ea61d-dca6-4ec8-94d9-f5f632878764,NaT,NaT,siet826,[],{},CTS Topic-Wise QP 16.10.2020
504234,33c5f796-d0a5-4208-8aa3-70b57d7c42c5,699d660e-9bb4-4d8b-9d06-85133654e9e6,<p></p>,Medium,mcq_single_correct,original_question,None,None,None,None,None,None,2018-09-27 12:29:14,511d98eb-361f-4bdb-8cc9-f0f62cb96edd,NaT,NaT,spark327,[],{},Sequence


In [16]:
# Storing the data where question is present:
cleaned_data1 = whole_data_notDeleted.drop(index=no_ques_data.index,axis=0)

In [17]:
# Calling the class object
duplicate = duplicate_v1()

In [18]:
# removing the html tags from the data and storing the data:
cleaned_data2 = duplicate.clean_data(cleaned_data1)

Data cleaned...
Time taken :: 37.16505789756775


In [19]:
# remove the nan created while removing the html tags
cleaned_data3 = duplicate.clean_nan(cleaned_data2)
cleaned_data3.shape

(8439211, 21)

In [20]:
#inserting column to the data:
duplicate.insert_col(cleaned_data3,18,"cleaned_mcq_questions_options",value="")

In [21]:
# transforming the cleaned data

transformed_data = duplicate.transform_data(cleaned_data3)
transformed_data.shape

(8439211, 22)

In [22]:
# Filtering out the data whose duplicates exists:
filtered_data = duplicate.filter_duplicate(transformed_data)
filtered_data.shape

(6718170, 24)

In [23]:
# collecting the index of duplicates of a random single question
num = 70
idx = duplicate.find_dup_idx(filtered_data,cleaned_data3["clean_question_data"].iloc[num])
print(f"""Number of duplicate for question :: {cleaned_data3["clean_question_data"].iloc[num]}\n is {len(idx)}""")

Number of duplicate for question :: find the number of diagonals in polygon of 15 sides.
 is 163


In [24]:
# fetching the variations of the above question
duplicate.variations(filtered_data=filtered_data,idx=idx)

filtered_data :: 163
q_type :: 163
q_sub :: 163
q_topic :: 163
q_sub_topic :: 163
Variation found...
Time taken :: 0.027292251586914062


,index,qb_id,q_id,question_data,clean_question_data,manual_difficulty,question_type,imported,subject_id,topic_id,...,createdAt,school_id,q_deletedAt,qb_deletedAt,school_code,cleaned_mcq_questions_options,mcq_questions_options,mcq_questions_answer,qb_name,duplicate
103591,209948,c6bda7b0-543c-4022-bc03-5f501752c2bb,7731b90d-35f5-4773-9802-d12ee0e6c591,<p>Find the number of diagonals in polygon of ...,find the number of diagonals in polygon of 15 ...,Easy,mcq_single_correct,unlimited_purchase,3639c0f6-9b40-43d2-aafc-90150e1cbf7b,7b5e555e-5199-4955-8462-292c6a975daa,...,2020-10-07 08:56:31,3f676f92-09af-4a12-8d03-695a5d516df0,NaT,NaT,pvpsit551,,"[{""text"":""<p>105</p>"",""media"":""""},{""text"":""<p>...","{""args"":[""<p>90</p>""],""partial"":[]}",Cognizant V1_4,True
104230,210816,a087b6e8-34e9-487f-b61f-bb894cea8997,64a3ea7a-1c02-4157-86d8-98fee5c256ec,<p>Find the number of diagonals in polygon of ...,find the number of diagonals in polygon of 15 ...,Easy,mcq_single_correct,unlimited_purchase,3639c0f6-9b40-43d2-aafc-90150e1cbf7b,7b5e555e-5199-4955-8462-292c6a975daa,...,2020-05-12 07:19:09,1e219353-c3bf-48b0-9825-e17ff94ec4f2,NaT,NaT,abes819,,"[{""text"":""<p>105</p>"",""media"":""""},{""text"":""<p>...","{""args"":[""<p>90</p>""],""partial"":[]}",Daily Aptitude Challenge 24,True
110283,220612,6376c5b7-0c3b-4871-970f-25be1782478d,bdf6c06a-f316-4008-89ec-184cf6afdec2,<p>Find the number of diagonals in polygon of ...,find the number of diagonals in polygon of 15 ...,Easy,mcq_single_correct,unlimited_purchase,3639c0f6-9b40-43d2-aafc-90150e1cbf7b,7b5e555e-5199-4955-8462-292c6a975daa,...,2020-09-10 07:48:27,6535d3dc-bbc6-4efc-bba3-6f0a2db5bea7,NaT,NaT,nandha552,,"[{""text"":""<p>105</p>"",""media"":""""},{""text"":""<p>...","{""args"":[""<p>90</p>""],""partial"":[]}",AMCAT Practice Test 4,True
178023,329798,ca8ec2aa-d3c5-47f0-9634-a467da5ca4b0,83578c4a-f765-4037-8824-a63480409e33,<p>Find the number of diagonals in polygon of ...,find the number of diagonals in polygon of 15 ...,Easy,mcq_single_correct,unlimited_purchase,3639c0f6-9b40-43d2-aafc-90150e1cbf7b,7b5e555e-5199-4955-8462-292c6a975daa,...,2020-05-31 15:38:52,61ae87ac-7207-4f2c-a511-19d2230ad59e,NaT,NaT,psgcas355,,"[{""text"":""<p>105</p>"",""media"":""""},{""text"":""<p>...","{""args"":[""<p>90</p>""],""partial"":[]}",Bootcamp Quants Practice 1,True
195279,356377,5cec3ac1-12dc-4ecf-842c-8c220fe2c845,4295f6d1-0c19-43b5-81ff-6ec8136713a2,<p>Find the number of diagonals in polygon of ...,find the number of diagonals in polygon of 15 ...,Easy,mcq_single_correct,unlimited_purchase,3639c0f6-9b40-43d2-aafc-90150e1cbf7b,7b5e555e-5199-4955-8462-292c6a975daa,...,2020-05-12 08:11:06,2f547e66-81f8-4082-934b-5f798612f639,NaT,NaT,vcet972,,"[{""text"":""<p>105</p>"",""media"":""""},{""text"":""<p>...","{""args"":[""<p>90</p>""],""partial"":[]}",Daily Aptitude Challenge 24,True
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6370646,8897967,1dd32622-e85f-4cea-841c-8c4b11b8a749,9e3faf71-1351-43a4-8ddc-d85b3422cffa,<p>Find the number of diagonals in polygon of ...,find the number of diagonals in polygon of 15 ...,Easy,mcq_single_correct,unlimited_purchase,3639c0f6-9b40-43d2-aafc-90150e1cbf7b,7b5e555e-5199-4955-8462-292c6a975daa,...,2021-07-28 13:30:11,0f65854e-9655-44fb-8d4b-a15d26a52aec,NaT,NaT,srit508,,"[{""text"":""<p>105</p>"",""media"":""""},{""text"":""<p>...","{""args"":[""<p>90</p>""],""partial"":[]}",CTS Mock Test 3,True
6396748,8925025,e6457c64-d0fd-4211-8f2f-ca1a77053927,f89d9a7a-fd84-4326-a5ec-34544dfc7ee5,<p>Find the number of diagonals in polygon of ...,find the number of diagonals in polygon of 15 ...,Easy,mcq_single_correct,unlimited_purchase,3639c0f6-9b40-43d2-aafc-90150e1cbf7b,7b5e555e-5199-4955-8462-292c6a975daa,...,2021-07-24 15:26:11,1ad6d356-9877-497e-a07c-2e0cb6251a37,NaT,NaT,gprec748,,"[{""text"":""<p>105</p>"",""media"":""""},{""text"":""<p>...","{""args"":[""<p>90</p>""],""partial"":[]}",CTS Mock Test 3,True
6452903,8983958,4d9

In [25]:
# duplicated data:
dup = duplicate.variations(filtered_data=filtered_data,idx=idx)

filtered_data :: 163
q_type :: 163
q_sub :: 163
q_topic :: 163
q_sub_topic :: 163
Variation found...
Time taken :: 0.002485990524291992


In [26]:
# cleaning the data answer data:
ans_cleaned_data3 = duplicate.clean_answer_data(dup)

Column present....
0
Answer Data cleaned...
Time taken :: 0.09773135185241699


/opt/conda/lib/python3.7/site-packages/pandas/core/indexing.py:1732: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_single_block(indexer, value, name)


In [27]:
ans_cleaned_data3.head()

,index,qb_id,q_id,question_data,clean_question_data,manual_difficulty,question_type,imported,subject_id,topic_id,...,createdAt,school_id,q_deletedAt,qb_deletedAt,school_code,cleaned_mcq_questions_options,mcq_questions_options,mcq_questions_answer,qb_name,duplicate
103591,209948,c6bda7b0-543c-4022-bc03-5f501752c2bb,7731b90d-35f5-4773-9802-d12ee0e6c591,<p>Find the number of diagonals in polygon of ...,find the number of diagonals in polygon of 15 ...,Easy,mcq_single_correct,unlimited_purchase,3639c0f6-9b40-43d2-aafc-90150e1cbf7b,7b5e555e-5199-4955-8462-292c6a975daa,...,2020-10-07 08:56:31,3f676f92-09af-4a12-8d03-695a5d516df0,NaT,NaT,pvpsit551,"[105, 30, 90, 60]","[{""text"":""<p>105</p>"",""media"":""""},{""text"":""<p>...","{""args"":[""<p>90</p>""],""partial"":[]}",Cognizant V1_4,True
104230,210816,a087b6e8-34e9-487f-b61f-bb894cea8997,64a3ea7a-1c02-4157-86d8-98fee5c256ec,<p>Find the number of diagonals in polygon of ...,find the number of diagonals in polygon of 15 ...,Easy,mcq_single_correct,unlimited_purchase,3639c0f6-9b40-43d2-aafc-90150e1cbf7b,7b5e555e-5199-4955-8462-292c6a975daa,...,2020-05-12 07:19:09,1e219353-c3bf-48b0-9825-e17ff94ec4f2,NaT,NaT,abes819,"[105, 30, 90, 60]","[{""text"":""<p>105</p>"",""media"":""""},{""text"":""<p>...","{""args"":[""<p>90</p>""],""partial"":[]}",Daily Aptitude Challenge 24,True
110283,220612,6376c5b7-0c3b-4871-970f-25be1782478d,bdf6c06a-f316-4008-89ec-184cf6afdec2,<p>Find the number of diagonals in polygon of ...,find the number of diagonals in polygon of 15 ...,Easy,mcq_single_correct,unlimited_purchase,3639c0f6-9b40-43d2-aafc-90150e1cbf7b,7b5e555e-5199-4955-8462-292c6a975daa,...,2020-09-10 07:48:27,6535d3dc-bbc6-4efc-bba3-6f0a2db5bea7,NaT,NaT,nandha552,"[105, 30, 90, 60]","[{""text"":""<p>105</p>"",""media"":""""},{""text"":""<p>...","{""args"":[""<p>90</p>""],""partial"":[]}",AMCAT Practice Test 4,True
178023,329798,ca8ec2aa-d3c5-47f0-9634-a467da5ca4b0,83578c4a-f765-4037-8824-a63480409e33,<p>Find the number of diagonals in polygon of ...,find the number of diagonals in polygon of 15 ...,Easy,mcq_single_correct,unlimited_purchase,3639c0f6-9b40-43d2-aafc-90150e1cbf7b,7b5e555e-5199-4955-8462-292c6a975daa,...,2020-05-31 15:38:52,61ae87ac-7207-4f2c-a511-19d2230ad59e,NaT,NaT,psgcas355,"[105, 30, 90, 60]","[{""text"":""<p>105</p>"",""media"":""""},{""text"":""<p>...","{""args"":[""<p>90</p>""],""partial"":[]}",Bootcamp Quants Practice 1,True
195279,356377,5cec3ac1-12dc-4ecf-842c-8c220fe2c845,4295f6d1-0c19-43b5-81ff-6ec8136713a2,<p>Find the number of diagonals in polygon of ...,find the number of diagonals in polygon of 15 ...,Easy,mcq_single_correct,unlimited_purchase,3639c0f6-9b40-43d2-aafc-90150e1cbf7b,7b5e555e-5199-4955-8462-292c6a975daa,...,2020-05-12 08:11:06,2f547e66-81f8-4082-934b-5f798612f639,NaT,NaT,vcet972,"[105, 30, 90, 60]","[{""text"":""<p>105</p>"",""media"":""""},{""text"":""<p>...","{""args"":[""<p>90</p>""],""partial"":[]}",Daily Aptitude Challenge 24,True


In [28]:
ans_cleaned_data3.iloc[np.where(ans_cleaned_data3["cleaned_mcq_questions_options"].duplicated() == True)]

,index,qb_id,q_id,question_data,clean_question_data,manual_difficulty,question_type,imported,subject_id,topic_id,...,createdAt,school_id,q_deletedAt,qb_deletedAt,school_code,cleaned_mcq_questions_options,mcq_questions_options,mcq_questions_answer,qb_name,duplicate
104230,210816,a087b6e8-34e9-487f-b61f-bb894cea8997,64a3ea7a-1c02-4157-86d8-98fee5c256ec,<p>Find the number of diagonals in polygon of ...,find the number of diagonals in polygon of 15 ...,Easy,mcq_single_correct,unlimited_purchase,3639c0f6-9b40-43d2-aafc-90150e1cbf7b,7b5e555e-5199-4955-8462-292c6a975daa,...,2020-05-12 07:19:09,1e219353-c3bf-48b0-9825-e17ff94ec4f2,NaT,NaT,abes819,"[105, 30, 90, 60]","[{""text"":""<p>105</p>"",""media"":""""},{""text"":""<p>...","{""args"":[""<p>90</p>""],""partial"":[]}",Daily Aptitude Challenge 24,True
110283,220612,6376c5b7-0c3b-4871-970f-25be1782478d,bdf6c06a-f316-4008-89ec-184cf6afdec2,<p>Find the number of diagonals in polygon of ...,find the number of diagonals in polygon of 15 ...,Easy,mcq_single_correct,unlimited_purchase,3639c0f6-9b40-43d2-aafc-90150e1cbf7b,7b5e555e-5199-4955-8462-292c6a975daa,...,2020-09-10 07:48:27,6535d3dc-bbc6-4efc-bba3-6f0a2db5bea7,NaT,NaT,nandha552,"[105, 30, 90, 60]","[{""text"":""<p>105</p>"",""media"":""""},{""text"":""<p>...","{""args"":[""<p>90</p>""],""partial"":[]}",AMCAT Practice Test 4,True
178023,329798,ca8ec2aa-d3c5-47f0-9634-a467da5ca4b0,83578c4a-f765-4037-8824-a63480409e33,<p>Find the number of diagonals in polygon of ...,find the number of diagonals in polygon of 15 ...,Easy,mcq_single_correct,unlimited_purchase,3639c0f6-9b40-43d2-aafc-90150e1cbf7b,7b5e555e-5199-4955-8462-292c6a975daa,...,2020-05-31 15:38:52,61ae87ac-7207-4f2c-a511-19d2230ad59e,NaT,NaT,psgcas355,"[105, 30, 90, 60]","[{""text"":""<p>105</p>"",""media"":""""},{""text"":""<p>...","{""args"":[""<p>90</p>""],""partial"":[]}",Bootcamp Quants Practice 1,True
195279,356377,5cec3ac1-12dc-4ecf-842c-8c220fe2c845,4295f6d1-0c19-43b5-81ff-6ec8136713a2,<p>Find the number of diagonals in polygon of ...,find the number of diagonals in polygon of 15 ...,Easy,mcq_single_correct,unlimited_purchase,3639c0f6-9b40-43d2-aafc-90150e1cbf7b,7b5e555e-5199-4955-8462-292c6a975daa,...,2020-05-12 08:11:06,2f547e66-81f8-4082-934b-5f798612f639,NaT,NaT,vcet972,"[105, 30, 90, 60]","[{""text"":""<p>105</p>"",""media"":""""},{""text"":""<p>...","{""args"":[""<p>90</p>""],""partial"":[]}",Daily Aptitude Challenge 24,True
200535,364883,e2caf577-f274-4cee-a28d-a5da6bdcb5a9,204fa0c5-5106-499f-99f9-ae6306e1c490,<p>Find the number of diagonals in polygon of ...,find the number of diagonals in polygon of 15 ...,Easy,mcq_single_correct,unlimited_purchase,3639c0f6-9b40-43d2-aafc-90150e1cbf7b,7b5e555e-5199-4955-8462-292c6a975daa,...,2021-07-26 06:19:28,aecc61e6-eb9a-4eee-8ae8-35d7529d00e6,NaT,NaT,srec694,"[105, 30, 90, 60]","[{""text"":""<p>105</p>"",""media"":""""},{""text"":""<p>...","{""args"":[""<p>90</p>""],""partial"":[]}",CTS Mock Test 3,True
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6370646,8897967,1dd32622-e85f-4cea-841c-8c4b11b8a749,9e3faf71-1351-43a4-8ddc-d85b3422cffa,<p>Find the number of diagonals in polygon of ...,find the number of diagonals in polygon of 15 ...,Easy,mcq_single_correct,unlimited_purchase,3639c0f6-9b40-43d2-aafc-90150e1cbf7b,7b5e555e-5199-4955-8462-292c6a975daa,...,2021-07-28 13:30:11,0f65854e-9655-44fb-8d4b-a15d26a52aec,NaT,NaT,srit508,"[105, 30, 90, 60]","[{""text"":""<p>105</p>"",""media"":""""},{""text"":""<p>...","{""args"":[""<p>90</p>""],""partial"":[]}",CTS Mock Test 3,True
6396748,8925025,e6457c64-d0fd-4211-8f2f-ca1a77053927,f89d9a7a-fd84-4326-a5ec-34544dfc7ee5,<p>Find the number of diagonals in polygon of ...,find the number of diagonals in polygon of 15 ...,Easy,mcq_single_correct,unlimited_purchase,3639c0f6-9b40-43d2-aafc-90150e1cbf7b,7b5e555e-5199-4955-8462-292c6a975daa,...,2021-07-24 15:26:11,1ad6d356-9877-497e-a07c-2e0cb6251a37,NaT,NaT,gprec748,"[105, 30, 90, 60]","[{""text"":""<

# Conclusion:

The code is also able to handle the whole data that include every question type present. Given a high-end machine, the code takes minimal time to execute and gives high precission results. We have also witnessed above that only mcq_single_correct type of question which takes 92% of the whole data. We can run the same above code through the data that are deleted which accounts for almost 8 lakh records if there comes a necessity. 